In [6]:
from __future__ import print_function
import argparse
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from dotmap import DotMap

import torch.nn.functional as F


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args=DotMap()
args.batch_size=128
args.saveInt=1
args.epochs=20
args.cuda=True
args.seed=1
args.log_interval=10
args.outf='./vae_mnist'
args.adv_out='./vae_adv'
args.baseline_adv_out='./vae_adv_baseline'
args.cuda = not args.no_cuda and torch.cuda.is_available()
try:
    os.makedirs(args.outf)
except OSError:
    pass
try:
    os.makedirs(args.adv_out)
except OSError:
    pass
try:
    os.makedirs(args.baseline_adv_out)
except OSError:
    pass


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)



kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)



class VAE_E(nn.Module):
    def __init__(self):
        super(VAE_E, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
          std = logvar.mul(0.5).exp_()
          eps = Variable(std.data.new(std.size()).normal_())
          return eps.mul(std).add_(mu)
        else:
          return mu

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

class VAE_D(nn.Module):
    def __init__(self):
        super(VAE_D, self).__init__()
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))
    
    
class Net_Classifier(nn.Module):
    def __init__(self):
        super(Net_Classifier, self).__init__()
        self.fc1 = nn.Linear(20, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x)

model_E = VAE_E()
model_D = VAE_D()
net_classifier=Net_Classifier()    

if args.cuda:
    model_E.cuda()
    model_D.cuda()
    net_classifier.cuda()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= args.batch_size * 784

    return BCE + KLD


optimizer_E = optim.Adam(model_E.parameters(), lr=1e-3)
optimizer_D = optim.Adam(model_D.parameters(), lr=1e-3)
optimizer_classifier = optim.Adam(net_classifier.parameters(), lr=1e-3)     

def train(epoch):
    model_E.train()
    model_D.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data = data.cuda()
            target=target.cuda()
        data, target = Variable(data), Variable(target)    
        optimizer_E.zero_grad()
        optimizer_D.zero_grad()
        z, mu, logvar = model_E(data)
        recon_batch=model_D(z)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward(retain_graph=True)
        train_loss += loss.data[0]
        
        optimizer_E.step()
        optimizer_D.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

def test(epoch):
    model_E.eval()
    model_D.eval()
    test_loss = 0
    for i, (data, target) in enumerate(test_loader):
        if args.cuda:
            data = data.cuda()
            target=target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)        
        z, mu, logvar = model_E(data)
        recon_batch = model_D(z)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if i == 0:
          n = min(data.size(0), 8)
          comparison = torch.cat([data[:n],
                                  recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
          save_image(comparison.data.cpu(),
                     args.outf+'/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    test_loss /= len(test_loader.dataset)

for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, 20))
    if args.cuda:
       sample = sample.cuda()
    sample = model_D(sample).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
               args.outf+'/sample_' + str(epoch) + '.png')

    if epoch%args.saveInt == 0 and epoch!=0:
        torch.save(model_E.state_dict(), '%s/vae_e_epoch_%d.pth' % (args.outf, epoch))
    if epoch%args.saveInt == 0 and epoch!=0:
        torch.save(model_D.state_dict(), '%s/vae_d_epoch_%d.pth' % (args.outf, epoch))   

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005476
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.003030
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002348
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002137
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002123
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002144
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.001994
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002092
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001921
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001873
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001843
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001817
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001721
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001607
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001595
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001616
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001619
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001611
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001590
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001574


Process Process-86:
Traceback (most recent call last):
  File "/home/torrvision/miniconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/home/torrvision/miniconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/torrvision/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/torrvision/miniconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    return recv()
Exception in thread Thread-89:
Traceback (most recent call last):
  File "/home/torrvision/miniconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/torrvision/miniconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/torrvision/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 50, in _pin_me

KeyboardInterrupt: 

In [4]:
def train_classifier(epoch):
    net_classifier.train()
    model_E.eval()
    model_D.eval()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer_classifier.zero_grad()
        z, mu, logvar = model_E(data)
        output = net_classifier(z)
        train_loss = F.nll_loss(output, target)
        train_loss.backward()
        optimizer_classifier.step()
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss.data[0] / len(train_loader.dataset)))
    
def test_classifier():
    net_classifier.eval()
    model_E.eval()
    model_D.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        z, mu, logvar = model_E(data)
        output = net_classifier(z)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, args.epochs + 1):
    train_classifier(epoch)
    test_classifier()

====> Epoch: 1 Average loss: 0.0000

Test set: Average loss: 0.1897, Accuracy: 9481/10000 (95%)

====> Epoch: 2 Average loss: 0.0000

Test set: Average loss: 0.1532, Accuracy: 9572/10000 (96%)

====> Epoch: 3 Average loss: 0.0000

Test set: Average loss: 0.1309, Accuracy: 9624/10000 (96%)

====> Epoch: 4 Average loss: 0.0000

Test set: Average loss: 0.1166, Accuracy: 9667/10000 (97%)

====> Epoch: 5 Average loss: 0.0000

Test set: Average loss: 0.1066, Accuracy: 9694/10000 (97%)

====> Epoch: 6 Average loss: 0.0000

Test set: Average loss: 0.1000, Accuracy: 9712/10000 (97%)

====> Epoch: 7 Average loss: 0.0000

Test set: Average loss: 0.0945, Accuracy: 9717/10000 (97%)

====> Epoch: 8 Average loss: 0.0000

Test set: Average loss: 0.0920, Accuracy: 9727/10000 (97%)

====> Epoch: 9 Average loss: 0.0000

Test set: Average loss: 0.0901, Accuracy: 9733/10000 (97%)

====> Epoch: 10 Average loss: 0.0000

Test set: Average loss: 0.0861, Accuracy: 9734/10000 (97%)

====> Epoch: 11 Average loss:

In [12]:
# Generate Adversarial Samples for the VAE based Classifier
import random
import numpy as np
i=0

for data_orig,target_orig in test_loader:
    for ind in range(data_orig.size()[0]):
        data, target = data_orig[ind].unsqueeze(1), target_orig.unsqueeze(1)[ind]
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        z, mu, logvar = model_E(data)
        recon_batch=model_D(z)
        output = net_classifier(z)
        pred = output.data.max(1, keepdim=True)
        res=pred[1].cpu()[0][0]
        adv_targets=set(list(range(0,10)))-set([res])
        adv_target=random.sample(adv_targets, 1)[0]
        #print(output)


        # Generate Adversarial Image 
        x = Variable(data.data, requires_grad=True)
        y_true=target
        SoftmaxWithXent = nn.CrossEntropyLoss()
        # Forward pass
        z, mu , logvar=model_E(x)
        outputs = net_classifier(z)
        loss = SoftmaxWithXent(outputs, y_true)
        loss.backward() # obtain gradients on x

        # Add perturbation
        epsilon = 0.1 
        x_grad   = torch.sign(x.grad.data)
        x_adversarial = torch.clamp(x.data + epsilon * x_grad, 0, 1) 

        if args.cuda:
            x_adversarial=x_adversarial.cuda()
        x_adversarial=Variable(x_adversarial)    
        # Classification after optimization  
        z_adv, mu_adv , logvar_adv=model_E(x_adversarial)
        y_pred_adversarial = np.argmax(net_classifier(z_adv).cpu().data.numpy())

        if res!=y_pred_adversarial:
            print(y_true)
            print(res)
            print(y_pred_adversarial)

            save_image(data.data.cpu(),'%s/test_%d.png' % (args.adv_out,i))
            save_image(recon_batch.view(1,28,28).data.cpu(),'%s/recon_%d.png' % (args.adv_out,i))
            save_image(x_adversarial.view(1,28,28).data.cpu(),'%s/adv_img_%d.png' % (args.adv_out,i))
            i+=1
    
print(i*100.0/len(test_loader.dataset))
print(len(test_loader.dataset))

Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

9
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variab

Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

9
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
1
Variab

Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

1
4
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
4
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
7
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
8
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
9
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variab

Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variab

Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

8
1
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

9
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
9
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
9
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variab

...

Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
4
Variab

Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

8
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
1
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variab

Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
8
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
9
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variab

Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
1
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 5
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
9
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variab

Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 7
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
7
Variable containing:
 4
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variable containing:
 0
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 3
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 8
[torch.cuda.LongTensor of size 1 (GPU 0)]

5
4
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
5
Variable containing:
 2
[torch.cuda.LongTensor of size 1 (GPU 0)]

7
4
Variable containing:
 9
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
5
Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]

4
7
Variab

In [15]:
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
x = np.linspace(0, 5, 10, endpoint=False)
y = multivariate_normal.pdf(x, mean=2.5, cov=0.5); y
plt.plot(x, y)